In [0]:
ROOT='/content/drive/My Drive/dl/推荐评论/'
BERT_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/新建文件夹/bert-base-chinese/'
BERT_TOK_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/bert-base-chinese-vocab.txt'
MAX_LEN=22

In [3]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install scikit-learn==0.20.2

In [4]:
import sklearn
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertModel, BertTokenizer
root='/content/drive/My Drive/google-quest-challenge/'
import numpy as np
import time
import tqdm

import math
import scipy
import torch.nn.functional as F
import random
import transformers, sys, os, gc
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers.modeling_bert import BertPreTrainedModel 
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_curve 
import numpy as np

In [0]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(42)

In [0]:
labels,train_data=[],[]
test=[]
weight=[]
lenth = []

In [0]:
with open (ROOT+'data/train.txt', 'r') as f:
  for line in f:
    label,text = line.split('\t')
    labels.append(int(label))
    train_data.append(text.replace('\n',''))

In [0]:
for label in labels:
  if label == 1:
    weight.append(3)
  else:
    weight.append(1)


In [9]:
tokenizer=BertTokenizer.from_pretrained(BERT_TOK_ROOT)
train_idx = []
train_mask = []
train_segment = []

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
for line in range(len(train_data)):
  token = tokenizer.tokenize(train_data[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  lenth.append(len(token))
  if len(token)> MAX_LEN:
      raise IndexError("Token length more than max seq length!")

  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  train_idx.append(idx)
  train_mask.append(mask)
  train_segment.append(sent)

# **统计特征**

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [0]:
lenths = np.array(lenth)
lenths=lenths.reshape(-1,1)

In [0]:
use = lenths

# **Dataset**

In [0]:
class EvalDataset(Dataset):
  def __init__(self,train_idx,train_mask,train_segment,use,label=None):
    self.train_idx = torch.tensor(train_idx,dtype=torch.int).long()
    self.train_mask = torch.tensor(train_mask,dtype=torch.int).long()
    self.train_segment = torch.tensor(train_segment,dtype=torch.int).long()
    self.use = torch.tensor(use , dtype=torch.float32)
    self.label = None
    if label is not None:
      self.label = torch.tensor(label,dtype=torch.float32)
  def __len__(self):
    return len(self.train_idx)
  def __getitem__(self,item):
    if self.label is not None:

      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'train_segment' :self.train_segment[item],
        'use':self.use[item],
        'label' : self.label[item]
    }
    else:
      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'use' : self.use[item],
        'train_segment' :self.train_segment[item],
      }
    return {key: value for key, value in output.items()}

In [0]:
def get_ids(x,y):
  z=[]
  for i in y:
    z.append(x[i])
  return z

# MODEL

In [0]:
class CLSbert(BertPreTrainedModel):
  def __init__(self,config,use_shape):
    super(CLSbert,self).__init__(config)
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.3)
    
    self.softmax = nn.Softmax(dim=1)
    self.pre = nn.Linear(2*768+use_shape,1)
    self.linear1 = nn.Linear(2*768+use_shape,2*768+use_shape)
    self.dropoutlist = nn.ModuleList([nn.Dropout(0.7) for _ in range(7)])

    self.ln = nn.LayerNorm(2*768+use_shape)
    self.d_model = 768
    self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
    nn.init.xavier_normal_(self.wt)
  def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
    outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
    w=outputs[2] 

    batch_size1,len_seq1,e_s1=w[1].size()
    bert_output1 = w[-1][0][int(use[0].item())-1].unsqueeze(0) #0个batch
    for i in range(1,batch_size1):
        bert_output1=torch.cat((bert_output1,w[-1][i][int(use[0].item())-1].unsqueeze(0)),dim=0)
    bert_output=w[0][:,0].reshape((-1,1,self.d_model)).to(device).clone()
    for i in range(1,13):
        bert_output=torch.cat((bert_output,w[i][:,0].reshape((-1,1,self.d_model)).to(device)),dim=1)
    bert_output=bert_output.view(batch_size1,13,-1)
    bert_output=bert_output.permute(0,2,1)
    bert_output=torch.matmul(bert_output,self.softmax(self.dropout(self.wt)))
    bert_output=torch.cat((bert_output.view(batch_size1,e_s1),use),dim=-1)
    bert_output=torch.cat((bert_output,bert_output1),dim=-1)
    l1=(self.linear1((self.dropoutlist[0]((bert_output)))))
    l2=(self.linear1((self.dropoutlist[1]((bert_output)))))
    l3=(self.linear1((self.dropoutlist[2]((bert_output)))))
    l4=(self.linear1((self.dropoutlist[3]((bert_output)))))
    l5=(self.linear1((self.dropoutlist[4]((bert_output)))))
    l6=(self.linear1((self.dropoutlist[5]((bert_output)))))
    l7=(self.linear1((self.dropoutlist[6]((bert_output)))))
    l=(l1+l2+l3+l4+l5+l6+l7)/7

    logits = self.pre(l)
    return logits

In [0]:
bert_model_config = BERT_ROOT+'/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 1
bert_config.output_hidden_states=True


In [0]:
def my_loss(input,target,weight=None):
  input=torch.sigmoid(input)
  if weight is not None:
    loss =torch._C._nn.binary_cross_entropy(input,target)
    loss*=weight
    return loss.sum()
  else:
    loss = torch._C._nn.binary_cross_entropy(input,target)
    return loss.sum()

In [0]:
def train_model(train_loader ,train_shape, optimizer ,scheduler, criterion, config=None , key = True):
  model.train()
  acc = 0
  auc = 0
  avg_loss = 0
  optimizer.zero_grad()
  for idx, batch in tqdm_notebook(enumerate(train_loader),mininterval=2,desc='--Training',leave=False):
    input_ids = batch['train_idx'].to(device)
    input_mask = batch['train_mask'].to(device)
    input_segments = batch['train_segment'].to(device)
    label = batch['label'].to(device).view(-1)
    use = batch['use'].to(device)

    output_train = model(use=use,input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_mask,
                             token_type_ids = input_segments,)
    loss = criterion(output_train.view(-1),label)                      
    #loss = my_loss(output_train.view(-1),label,weight)
    avg_loss += loss.item()
    loss.backward()
    optimizer.step()
    scheduler.step()
    acc += ((torch.sigmoid(output_train.cpu().view(-1))>0.5).int()==label.cpu()).sum()
    #auc += sklearn.metrics.roc_auc_score( (label.cpu().int()),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())
    fgm.attack()
    output_train = model(use,input_ids = input_ids.long(),
                              labels = None,
                              attention_mask = input_mask,
                              token_type_ids = input_segments,)
    loss_adv = criterion(output_train.view(-1),label)     
    loss_adv.backward()
    fgm.restore()

    optimizer.step()
    scheduler.step()
    
    
    ema.update()
    optimizer.zero_grad()
    
  return avg_loss/train_shape,acc.item()/train_shape,auc/len(train_loader)

In [0]:
def valid_model(valid_loader ,val_shape,  criterion, batch_size ,config=None):
  model.eval()
  acc = 0
  auc = 0
  avg_loss = 0
  pred = np.zeros((val_shape))
  ema.apply_shadow()
  with torch.no_grad():
    for idx, batch in tqdm_notebook(enumerate(valid_loader),mininterval=2,desc='--Valing',leave=False):
      input_ids = batch['train_idx'].to(device)
      input_mask = batch['train_mask'].to(device)
      input_segments = batch['train_segment'].to(device)
      label = batch['label'].to(device).view(-1)
      use = batch['use'].to(device)
      output_train = model(use=use,input_ids = input_ids.long(),
                              labels = None,
                              attention_mask = input_mask,
                              token_type_ids = input_segments,
                              )
      loss = criterion(output_train.view(-1),label)
      #loss = my_loss(output_train.view(-1),label,weight)
      pred[idx*batch_size : (idx+1)*batch_size] = output_train.view(-1).detach().cpu().squeeze().numpy()
      avg_loss += loss.item()
      acc += ((torch.sigmoid(output_train.cpu().view(-1))>0.5).int()==label.cpu()).sum()
      if input_ids.size()[0]!=1:

        auc += sklearn.metrics.roc_auc_score( (label.cpu().int()),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())
  ema.restore()
  return avg_loss/val_shape,acc.item()/val_shape,auc/len(valid_loader),torch.sigmoid(torch.tensor(pred)).numpy()

In [0]:
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [0]:
class FGM():
  def __init__(self,model):
    self.model=model
    self.backup={}
  def attack(self,epsilon=0.5,emb_name='word_emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        self.backup[name]=param.data.clone()
        if (param.grad)== None:
          continue
        norm=torch.norm(param.grad)
        if norm!= 0 and not torch.isnan(norm):
          r_at=epsilon * param.grad/norm
          param.data.add_(r_at)
  def restore(self,emb_name='word_emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        assert name in self.backup
        param.data=self.backup[name]
    self.backup={}

In [0]:
BATCH_SIZE =70
NUM_EPOCH=5
device = torch.device('cuda')

In [0]:
floder = StratifiedKFold(n_splits=5,random_state=0,shuffle=False)
predict = np.zeros(len(train_idx))


In [0]:
 
 
def transfer_model(pretrained_file, model):

    pretrained_dict = torch.load(pretrained_file)  # get pretrained dict
    model_dict = model.state_dict()  # get model dict
    # 在合并前(update),需要去除pretrained_dict一些不需要的参数
    pretrained_dict = transfer_state_dict(pretrained_dict, model_dict)
    model_dict.update(pretrained_dict)  # 更新(合并)模型的参数
    model.load_state_dict(model_dict)
    return model
 
 
def transfer_state_dict(pretrained_dict, model_dict):

    # state_dict2 = {k: v for k, v in save_model.items() if k in model_dict.keys()}
    state_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict.keys():
            # state_dict.setdefault(k, v)
            state_dict[k] = v
        else:
            print("Missing key(s) in state_dict :{}".format(k))
    return state_dict
 
 


伪标签

In [0]:
wlabel_label=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_label_bin.pkl')
wlabel_train_idx = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_idx.pkl')
wlabel_train_mask = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_mask.pkl')
wlabel_train_segment = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_segment.pkl')
wlabel_train_use=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_lenth.pkl')

In [0]:
use_shape = use.shape[1]
use = list(use)

In [29]:
torch.cuda.get_device_name()

'Tesla P4'

In [30]:
for fold,(train, test) in enumerate(floder.split(train_idx,labels)):
  if fold ==3:
    continue
  t_idx = get_ids(train_idx , train)+wlabel_train_idx
  t_mask = get_ids(train_mask , train)+wlabel_train_mask
  t_segment = get_ids(train_segment , train)+wlabel_train_segment
  t_weight = get_ids(weight , train)
  print(wlabel_label.shape)
  t_label = (np.vstack((np.array(get_ids(labels , train)).reshape(-1,1),wlabel_label.reshape(-1,1))))

  t_use = (np.vstack((np.array(get_ids(use , train)),(wlabel_train_use).reshape(-1,1))))
  valid_idx = get_ids(train_idx , test)
  valid_mask = get_ids(train_mask , test)
  valid_segment = get_ids(train_segment , test)
  valid_weight = get_ids(weight , test)
  valid_label = get_ids(labels , test)
  valid_use = get_ids(use , test)
  train_shape = len(t_idx)
  valid_shape = len(valid_idx)
  train_set = EvalDataset(t_idx , t_mask , t_segment , t_use ,t_label)
  vla_set = EvalDataset(valid_idx , valid_mask , valid_segment ,valid_use ,valid_label)
  train_loader = DataLoader (train_set , batch_size=BATCH_SIZE,shuffle=True)
  valid_loader = DataLoader (vla_set , batch_size=BATCH_SIZE,shuffle=False)
  
  model = CLSbert.from_pretrained(BERT_ROOT,use_shape = use_shape,config = bert_config)
  #model = transfer_model('/content/drive/My Drive/dl/推荐评论/CNN_3/best_param_score1.pkl',model)
  model.zero_grad();
  model.to(device);
  fgm = FGM(model)
  ema=EMA(model,0.995)
  ema.register()
  params = list(model.named_parameters())
  
 

  optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, eps=4e-5,weight_decay=0.8)
  
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.05, num_training_steps= NUM_EPOCH*len(train_loader))
  criterion = nn.BCEWithLogitsLoss()
  best_avg_loss = -100
  best_score = -1
  best_param_score = None
  key = True
  freeze_layers= []
  freeze_layers1 = ['layer.1.','layer.2','layer.3']
  freeze_layers2 = ['layer.4','layer.5','layer.6']
  freeze_layers3 = ['layer.7','layer.8','layer.9']
  freeze_layers4 = ['layer.10','layer.11','layer.12']
  freeze = []
  freeze.append(freeze_layers1)
  freeze.append(freeze_layers2)
  freeze.append(freeze_layers3)
  freeze.append(freeze_layers4)

  for epoch in range(NUM_EPOCH):
    # if epoch !=0:
    #   freeze_layers+=freeze[epoch-1]
    #   print(freeze_layers)
    #   for name ,param in model.named_parameters():
    #       for ele  in freeze_layers:
    #           if ele in name:
    #             param.requires_grad_(False)
    # for name ,param in model.named_parameters():
    #   if param.requires_grad == False:
    #     print(name)
   
    torch.cuda.empty_cache()
    start_time = time.time()
    train_loss,train_acc,train_auc=train_model(train_loader ,train_shape, optimizer ,scheduler, criterion, config=None ,key = key)
    val_loss,val_acc,val_auc,pred=valid_model(valid_loader ,valid_shape,  criterion, BATCH_SIZE ,config=None)
    elapsed_time = time.time() - start_time
    print('epoch{}/{} , train_loss: {} ,train_acc:{} ,train_auc: {} \n \n val_loss :{} , val_acc :{} , val_auc:{} ,time: {}\n'.format(epoch+1,NUM_EPOCH,train_loss,train_acc,train_auc,val_loss,val_acc,val_auc,elapsed_time))
    if best_score < val_auc:
      best_score = val_auc
      best_param_score = model.state_dict()
      predict[test] = pred
      ema.apply_shadow()
      torch.save(best_param_score,'/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/'+'CLS/best_param_score_{}'.format(fold+1))
      torch.save(ema,'/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/'+'CLS/EMA_{}'.format(fold+1))
  del model, optimizer, criterion, scheduler
  torch.cuda.empty_cache()
  del valid_loader, train_loader, vla_set, train_set

(56366,)


epoch1/5 , train_loss: 0.0022727365461522498 ,train_acc:0.9365430492301019 ,train_auc: 0.0 
 
 val_loss :0.0030863683406578976 , val_acc :0.9069040924711028 , val_auc:0.9724234028774832 ,time: 738.03435587883



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CLSbert. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch2/5 , train_loss: 0.0011562273800914296 ,train_acc:0.9686402081977878 ,train_auc: 0.0 
 
 val_loss :0.0029165201035748047 , val_acc :0.9215870040612308 , val_auc:0.9755936188694012 ,time: 743.1309716701508



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CLSbert. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

epoch2/5 , train_loss: 0.0012536960564917719 ,train_acc:0.9648242659071522 ,train_auc: 0.0 
 
 val_loss :0.0030515138508193603 , val_acc :0.9227883713660519 , val_auc:0.9716970553851513 ,time: 744.2418050765991



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CLSbert. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch3/5 , train_loss: 0.0006217767779371145 ,train_acc:0.9848048925065421 ,train_auc: 0.0 
 
 val_loss :0.00341949513267636 , val_acc :0.9221631759924976 , val_auc:0.9700656122695581 ,time: 744.2254631519318



epoch4/5 , train_loss: 0.0008992299178227102 ,train_acc:0.9764483062732228 ,train_auc: 0.0 
 
 val_loss :0.003268750255770816 , val_acc :0.9196623944982807 , val_auc:0.9695363614004837 ,time: 745.5078592300415



epoch5/5 , train_loss: 0.0014395405035242843 ,train_acc:0.9607327193603886 ,train_auc: 0.0 
 
 val_loss :0.0030019763390154866 , val_acc :0.9221631759924976 , val_auc:0.9707832795765117 ,time: 745.7116959095001



In [32]:
sklearn.metrics.roc_auc_score(labels,predict)

0.8019882069147437